# Importing Libraries

in this part we will install all the necessary libraries on command prompt and then import the necessary functions from those libraries. 

In [ ]:
# importing all the necessary libraries

import pandas as pd



from numpy import mean

import numpy as np

import time



# step 1: preprocessing

from sklearn.impute import SimpleImputer # import some strategic imputer to fill in any missing values using mean

from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, Normalizer # scale all the values to one range to avoid any biasness (this bias is seen in mostly naive bayes and knn etc)



from sklearn.impute import KNNImputer # import some strategic imputer to fill missing values using KNN (finds the nearest neighbour and fills it with that value)



from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_classif, VarianceThreshold



from sklearn.decomposition import PCA

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder

from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge, Lasso



# step 2: data division

from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, GridSearchCV, ParameterGrid # to divide the code into train/test using a specific percentage or with/without replacement



# step 3: model

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

from xgboost import XGBRegressor



# step 4: displaying accuracy

from sklearn.metrics import roc_auc_score, accuracy_score # to display the accuracy of our tree

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer



# step 5: warning filter

import warnings

warnings.filterwarnings('ignore')


# Data Loading

data shall be loaded into variables as data sets using pandas and csv readers. they will be checked to see if they are loaded properly and will be loaded as 2 sets: train and test as per given in the kaggle data

In [ ]:
# lets load the training data set

# train_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\train\train.csv")
train_data = pd.read_csv(r"/kaggle/input/challenge2/train.csv")


# lets also check it by getting the first few rows of the data, there should be x1 - x78 and one target variable Y

train_data.head() 


In [ ]:
# lets load the test data

# test_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\test\test.csv")
test_data = pd.read_csv(r"/kaggle/input/challenge2/test.csv")


# check if the data has been loaded by getting the first 5 rows - there should be x1 - x78 and no target variable Y as this is test data

test_data.head() 


# Data Preprocessing

before we start processing this data and using algorithms, we will fix this data first, this is called data preprocessing

## split data into categorical and numerical

categorical will have one-hot and simple imputer of most frequent while numerical will have simple mean imputer and minmax scaler

In [ ]:
categorical_cols = train_data.select_dtypes(include=["object"]).columns

numerical_cols = train_data.select_dtypes(exclude=["object"]).drop(columns=['price_doc']).columns


In [ ]:
# num_transformer = Pipeline(steps=[

#     ("imputer", SimpleImputer(strategy="mean")),

#     ("scaler", StandardScaler())

# ])

# cat_transformer = Pipeline(steps=[

#     ("imputer", SimpleImputer(strategy="most_frequent")),

#     ("onehot", OneHotEncoder(handle_unknown="ignore"))

# ])


In [ ]:
# # Column transformer for preprocessing

# preprocessor = ColumnTransformer(

#     transformers=[

#         ("num", num_transformer, numerical_cols),

#         ("cat", cat_transformer, categorical_cols)

#     ]

# )


In [ ]:
# num_imputer = SimpleImputer(strategy="median")
# train_data[numerical_cols] = num_imputer.fit_transform(train_data[numerical_cols])
# test_data[numerical_cols] = num_imputer.transform(test_data[numerical_cols])

# impute with fill median
train_data[numerical_cols] = train_data[numerical_cols].fillna(train_data[numerical_cols].median())
test_data[numerical_cols] = test_data[numerical_cols].fillna(test_data[numerical_cols].median())


In [ ]:
scaler = StandardScaler()
train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])
test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])


In [ ]:
train_data = pd.get_dummies(train_data, columns=categorical_cols, drop_first=True)
test_data = pd.get_dummies(test_data, columns=categorical_cols, drop_first=True)
test_data = test_data.reindex(columns=train_data.columns, fill_value=0)
# test_data = test_data.drop(columns=['price_doc'], errors="ignore")


## correlation matrix

i tried getting the correlation matrix but apparently a 2000 columns matrix is very computationally expensive as it performs pairs for all. so dont run it. it takes too long and then fails. i ran for 5 minutes. 

In [ ]:
# # DONT RUN

# corr_matrix = train_data.corr()

# print(corr_matrix)


# variance filter

In [ ]:
# train_features = train_data.drop(columns=['price_doc'])


In [ ]:
# feature_variances = train_features.var(axis=0)

# print(feature_variances.describe())


In [ ]:
# from sklearn.feature_selection import VarianceThreshold



# selector = VarianceThreshold(threshold=0.01) 

# train_features_reduced = selector.fit_transform(train_features)

# print(train_features_reduced)

# print(train_features_reduced.shape)



# test_features = selector.transform(test_data)


# PCA

principal component analysis is applied

In [ ]:
# # plot an elbow graph to find the optimal number of components

# import matplotlib.pyplot as plt

# from sklearn.decomposition import PCA



# pca = PCA()

# pca.fit(train_features_reduced)  # Fit PCA to your dataset



# plt.plot(pca.explained_variance_ratio_)

# plt.xlabel('number of components')

# plt.ylabel('cumulative explained variance')

# pca.explained_variance_ratio_.round(3)

# cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# print("Cumulative explained variance: ", cumulative_variance)

# print(cumulative_variance)


In [ ]:
# print("Principal components shape:", pca.components_.shape)

# print("Principal components:\n", pca.components_)



# # Preserve the target variable

# train_copy = train_data.copy()  # Backup original data

# test_copy = test_data.copy()



# # Separate target variable and features

# train_features = train_data.drop(columns=['price_doc'])  # Features for PCA

# train_target = train_data['price_doc']                  # Target variable



# test_features = test_data.drop(columns=['price_doc'])    # Features for PCA



# # Apply PCA transformation

# pca = PCA(n_components=0.95)

# train_features_pca = pca.fit_transform(train_features)  # Fit and transform train data

# print("train transformed")

# test_features_pca = pca.transform(test_features)        # Transform test data



# print(train_features_pca.shape)

# print(test_features_pca.shape)


In [ ]:
# # Reconstruct train_data and test_data with PCA-transformed features and target variable

# train_data = pd.DataFrame(train_features_pca)

# train_data['price_doc'] = train_target.reset_index(drop=True)



# test_data = pd.DataFrame(test_features_pca)

# test_data['price_doc'] = test_copy['price_doc'].reset_index(drop=True)


In [ ]:
print("train shape: ", train_data.shape)

print("test shape: ", test_data.shape)


## Data Splitting - festures and targets

the data in train_data set is of x1 - x78 columns (79 variables) and one target variable (Y). we must split that data so that we can perform data preprocessing on the features variables (will be referred to as X).

In [ ]:
# so in X, it is ALL the columns EXCEPT the last column known as 'Y' (we can confirm this using the train_data.head() we did earlier) so we must get all columns and DROP only the 'y' column

X = train_data.drop(columns=['price_doc'])

X # lets display X and see what it is now


In [ ]:
# so as per our X output, we can see that number of columns in train_data is 79 and number of columns in X is 78 meaning we have successfully performed our removal of target variable

# now to get the target variable alone, we can just get it alone,

Y = train_data['price_doc']

Y # lets see what it is

# as per our Y output, we can see it is of one column and 246k rows which means we have successfully extracted the target variable column


# Filters

there are two types of filters to filter out columns/features:

- variance filter (a column which has same values throughout the column like all are sunny)

- correlation filter (two columns which are same like weight in kg and weight in pounds)

In [ ]:
# print("X : ", X.shape)

# print("test data : ", test_data_processed.shape)


In [ ]:
# variance filter

# ----------------------------- case  -----------------------------

# variance_filter = VarianceThreshold(threshold=0.001)  # Adjust the threshold if needed

# X = variance_filter.fit_transform(X)

# test_data_processed = variance_filter.fit_transform(test_data_processed)

X.shape


In [ ]:
# test_data_processed.shape


In [ ]:
# # correlation filter

# # ----------------------------- case  -----------------------------

# corr_matrix = pd.DataFrame(X).corr().abs()

# upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

# X = pd.DataFrame(X).drop(columns=to_drop)

# test_data_processed = pd.DataFrame(test_data_processed).drop(columns=to_drop)

X.shape


In [ ]:
# test_data_processed.shape


## Data Splitting - train and validate

now our test_data set is of rows with NO target variable whereas the train_data set is WITH target variable.

our rules in machine learning is that we must train half or 70% of the data and then we must check its accuracy using the remaining half or 30% of the data - we can only check accuracy IF we have the answers i.e. the target variable. 

So, what we need to do is, is split the train_data set into 2, by a 70% and 30% ratio. we train the model using the 70% and then test the model using the 30% and then use that model to predict the test_data set.

In [ ]:
# holdout method

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=2)


# functions

here we have defined functions like forward-backward selection, kbest selection & algorithm feature importance

In [ ]:
# forward backward selection

def fbselection(direction, sample_model, features, X, trainX, trainY, testX, test_data_processed):

    print("starting")

    selection = SequentialFeatureSelector(sample_model, direction=direction, n_features_to_select=features, scoring='roc_auc')

    return modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed)



def modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed):

    print("start extracting")

    trainX = selection.fit_transform(trainX, trainY)

    print("extracted, transforming")

    testX = selection.transform(testX)                                  # Ensure the test set is transformed similarly

    test_data_processed = selection.transform(test_data_processed)      # test data is also transformed

    X = selection.transform(X)                                          # full data transforming

    print("transformed")

    return sample_model, X, trainX, trainY, testX, test_data_processed



# kbest selection

def kbest(features, X, trainX, trainY, testX, test_data_processed):

    print("starting")

    selection = SelectKBest(score_func=f_classif, k=features)

    return modelSelector(None, selection, X, trainX, trainY, testX, test_data_processed)


In [ ]:
# feature importance function

def featureImportance(sample_model, features, X, trainX, trainY, testX, test_data_processed):

    print("fitting")

    

    # fit the model

    sample_model.fit(trainX, trainY)



    print("extracting features")



    # extract all the feature names from data

    importances = sample_model.feature_importances_

    feature_names = train_data_processed.drop(columns=['price_doc']).columns

    print(feature_names)



    # sort with respect to importance

    feature_importance_df = pd.DataFrame({

        'Feature': feature_names,

        'Importance': importances

    }).sort_values(by='Importance', ascending=False)



    # extract the top ones

    top_features = feature_importance_df['Feature'].head(features).values

    print(top_features)



    # change all data according to the top ones we have selected

    trainX = pd.DataFrame(trainX, columns=feature_names)[top_features]

    testX = pd.DataFrame(testX, columns=feature_names)[top_features]

    X = pd.DataFrame(X, columns=feature_names)[top_features]

    test_data_processed = pd.DataFrame(test_data_processed, columns=feature_names)[top_features]



    print("features extracted")

    

    # retrain the model

    sample_model.fit(trainX, trainY)



    print("features trained")

    

    return sample_model, X, trainX, trainY, testX, test_data_processed


## model intialization

here model is intialized

In [ ]:
# trainX = preprocessor.fit_transform(trainX)

# print("trainX completed")

# testX = preprocessor.transform(testX)

# print("testX completed")

# test_data = preprocessor.transform(test_data)

# print("test data completed")

# X = preprocessor.transform(X)

# print(X.shape)


In [ ]:
xgb_model =  XGBRegressor( max_depth=19, learning_rate=0.006, n_estimators=2400, subsample=0.88, reg_lambda=0.15, reg_alpha=0.85, device = 'cuda', verbose = 2, tree_method = 'gpu_hist', predictor = 'gpu_predictor' )


In [ ]:
# # apply feature selection here

xgb_model, X, trainX, trainY, testX, test_data = featureImportance(xgb_model, 100, X, trainX, trainY, testX, test_data)


In [ ]:
# Build the Neural Network
def build_nn(input_dim):
    model = Sequential()
    model.add(Dense(2048, activation="relu", input_dim=input_dim))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(512, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(256, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(16, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(8, activation="relu"))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer=Adam(learning_rate=0.0015), loss="mse")
    return model


In [ ]:
# Initialize the model

nn_model = build_nn(trainX.shape[1])



# Define callbacks
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True, verbose=1)

# Train the model
nn_model = nn_model.fit(
    trainX, trainY,
    validation_data=(testX, testY),
    epochs=250,
    batch_size=64,
    verbose=2,
    callbacks=[lr_scheduler, early_stopping]
)


In [ ]:
y_pred_scaled = nn_model.predict(testX).flatten()

y_pred = y_pred_scaled


In [ ]:
# apply feature selection here

# nn_model, X, trainX, trainY, testX, test_data = featureImportance(nn_model, 100, X, trainX, trainY, testX, test_data)


In [ ]:
print("X shape -> ", X.shape)

print("trainX shape -> ", trainX.shape)

print("testX shape -> ", testX.shape)

print("test_data_processed shape -> ", test_data.shape)


## model running

here we run the model

In [ ]:
y_pred_scaled = nn_model.predict(testX).flatten()

y_pred = y_pred_scaled


In [ ]:
# display the mean squared error of this prediction

mse = mean_squared_error(testY, y_pred)

print("Mean squared error: %.2f" % mse, "   ")



    # display the root mean squared error

rmse = np.sqrt(mse)  # Root Mean Squared Error

print("Root Mean squared error: %.2f" % rmse, "   ")


In [ ]:
# fit the model

nn_model.fit(trainX, trainY)


In [ ]:
# display information regarding the regression

# print("model score: ", nn_model.score(trainX, trainY))

# print("model coefficient: ", model.coef_)

# print("model intercept: ", model.intercept_)


In [ ]:
# compute this predictions metrics

def metrics(y_pred, testY):

    print("starting to compute metrics")

    

    # # display the accuracy of this prediction

    # accuracy = accuracy_score(testY, y_pred)

    # print("model accuracy = ", accuracy, "   ")



    # # now lets calculate the ROC AUC score according to this prediction

    # roc_score = roc_auc_score(testY, y_pred)

    # print("roc score = ", roc_score, "   ")



    # display the mean squared error of this prediction

    mse = mean_squared_error(testY, y_pred)

    print("Mean squared error: %.2f" % mse, "   ")



    # display the root mean squared error

    rmse = np.sqrt(mse)  # Root Mean Squared Error

    print("Root Mean squared error: %.2f" % rmse, "   ")



    # display the mean absolute error of this prediction

    mae = mean_absolute_error(testY, y_pred)

    print("Mean absolute error: %.2f" % mae, "   ")



    # display the coeffeicient of determination of this preduction

    r2_Score = r2_score(testY, y_pred)

    print("Coefficient of determination: %.2f" % r2_Score, "    ")


In [ ]:
# predict using this model USING PREDICT

y_pred = nn_model.predict(testX)

print("successfully predicted")

metrics(y_pred, testY)


In [ ]:
# # predict using thus model USING PREDICTPROBA

# y_pred_proba = model.predict_proba(testX)[:, 1]

# print("successfully predicted")

# metrics(y_pred_proba, testY)


## predict for test dataset

fit the model and predict for test dataset

In [ ]:
nn_model.fit(X, Y)


In [ ]:
# display information regarding the regression

# print("model score: ", nn_model.score(X, Y), "    ")

# print("model coefficient: ", model.coef_)

# print("model intercept: ", model.intercept_)


In [ ]:
X.columns


In [ ]:
test_data.columns


In [ ]:
y_test_pred_scaled = nn_model.predict(test_data.drop(columns=['price_doc'])).flatten()

y_test_pred = y_test_pred_scaled


## write into csv

now we write the predictions into the csv file

In [ ]:
# sample_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\sample_submission.csv")
sample_data = pd.read_csv(r"/kaggle/input/challenge2/sample_submission.csv")


sample_data['price_doc'] = y_test_pred

# sample_data = sample_data[]



# sample_data.to_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\lasso1.csv", index=False)
sample_data.to_csv(r"/kaggle/working/neuralnetwork1.csv")
sample_data


In [ ]:
nn_model
